## Les événements dans Dash

Il est bien de pouvoir interagir dans son tableau de bord. 
Cela peut être le fait de cliquer sur un bouton pour afficher un autre graphe, choisir les champs dans un menu, indiquer
si on préfère une échelle linéaire ou logarithmique, etc.

Pour tout cela il faut que le fait de cliquer sur un bouton, ou de passer au dessus d'un élément, déclanche un événement qui sera intercepté par le composant visé pour qu'il se mette à jour. 

Voici un exemple adapté du [chapitre 2 du tutorial](https://dash.plot.ly/getting-started-part-2) de Dash. Je mets dans mon tableau de bord 

* un champs d'entrée de texte
* du texte qui va intégrer d'une certaine facon ce que l'utilisateur aura écrit dans le champs

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id='my-free-form', value='', type='text'),
    html.Div(dcc.Markdown(id='my-text-displayed'))

])


@app.callback(
    Output(component_id='my-text-displayed', component_property='children'),
    [Input(component_id='my-free-form', component_property='value')],
    [State(component_id='my-text-displayed', component_property='children')]
)
def update_output_div(input_value, current_text): # values come from Input & State
    if current_text == None:
        return "Please type something"
    if len(input_value) == 0:
        current_text = ''
    return "%s  \n%s" % (current_text, input_value)

if __name__ == '__main__':
    app.run_server()

Dans la partie `layout` on trouve les 2 composants à savoir `dcc.Input` pour entrer son texte et `html.Div` pour afficher le résultat.

La partie intéressante ici est ce qu'on appelle le **`callback`**. Il s'agit d'une fonction de rappel qui sera 

* **exécutée lorsque un des `Input` est modifié** donc ici seulement le composant identifié par `my-free-form` (donc le composant `dcc.Input`)
* **utilisée par ce qui est mis en `Output`** donc ici le composant identifié par `my-text-displayed` (donc le componsant `html.Div`) 

Les `State`s étant des informations prises dans des composants qui seront utiles à l'exécution de la fonction.

Ces `Input`, `Output` et `State`
font partis du **décorateur  ` @app.callback` ** qui
permet de relier la fonction aux évéments déclanchés par les `Input` (notez que `Input` et `State` sont dans des listes
ce qui permet d'en avoir plusieurs). 

La fonction, `update_output_div` dans notre cas, 

* a autant d'arguments qu'il y a d'`Input` et de `State`, chaque variable étant la valeur indiquée par le champs `component_property`
* retourne ce qui ira dans le champs `component_property` indiqué l'`Output`. Une seule valeur de retour est possible.

Essayez de deviner de deviner ce qui se passe si j'entre
`**This** is *nice*` dans le `dcc.Input` (offrez-vous un verre si vous trouvez exactement la réponse !). Pour avoir la réponse
recopiez la cellule dans un fichier `toto.py`, lancez `python toto.py`puis regardez sur [127.0.0.1:8050](http://127.0.0.1:8050/).

### C'est tout !

À partir de là vous avez tous les éléments pour faire un tableau de bord. Tout le reste n'est que détail à savoir

* connaitre les composants possibles, cf [Dash HTML](https://dash.plot.ly/dash-html-components) et [Dash Core](https://dash.plot.ly/dash-core-components) avec une description des composants et des exemples d'usage
* lire la documentation des composants qui a des informations plus précise parfois (`html.Button?` par exemple)
* trouver les `component_property` à utiliser pour les `callback` (cachés dans les exemples ou dans le message d'erreur lorsqu'on met n'importe quoi)
* comprendre le style qui dérive du style de HTML, voir ce [tutorial de CSS](https://www.w3schools.com/css/default.asp) par exemple. Il y a néanmoins une différence importante : les mots clefs utilsent la notation dite camelCased et donc `'text-align'` en CSS devient `'textAlign'` dans la partie `style` de Dash


#### Attention aux variables globales

Le dernier gros détail concerne les variables globale. Dans l'exemple complet qu'on donne pour finir on utilise la
variable globale `df` pour le DataFrame qui stocke nos données. Mais si 

* le tableau de bord peut modifier cette variable (ce qui n'est pas le cas dans l'exemple suivant)
* plusieurs personnes sont connectée en même temps au tableau de bord 

alors une interaction effectuée par un utilisateur peut modifier le tableau de bord de tous les utilisateurs ce qui
n'est généralement pas le comportement voulu. Pour éviter cela regardez le tutorial sur le [partage de données entre fonctions de rappel](https://dash.plot.ly/sharing-data-between-callbacks).

## Un exemple plus complet

Pour finir voici le code du fichier qui génére un joli tableau de bord qui reprend l'évolution des pays dans le temps suivant
différents critère. Je vous suggère de 

* récupérer  <a href="data/dash.zip">cette archive</a> et de la décompresser,
* lancer l'exemple avec la commande `python dash_fertility.py`
* regarder le résultat ici : [http://127.0.0.1:8050/](http://127.0.0.1:8050) (127.0.0.1 est votre machine)
* faites des petites modifications dans le code (changer la couleur du fond d'un composant, ajouter plus de graphiques...), puis relancer `python dash_fertility.py` et regarder ce que cela donne.

In [ ]:
# %load dash_fertility.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly.graph_objs as go

START = 'Start'
STOP  = 'Stop'

def get_data():
    incomes = pd.read_excel('data/GDPpercapitaconstant2000US.xlsx', index_col=0).round()
    children = pd.read_excel('data/fertility.xlsx', index_col=0)
    population = pd.read_excel('data/population.xlsx', index_col=0).round()
    continent = pd.read_csv('data/countries_continent.csv', index_col=0)
    idx = (list(set(incomes.index) & set(children.index) 
        & set(population.index) & set(continent.index)))
    idx.sort()
    df = pd.concat({'incomes': incomes.loc[idx, '1960':'2010'],
            'population': population.loc[idx, '1960':'2010'],
            'children': children.loc[idx, '1960':'2010']}, axis='columns')
    df = df.swaplevel(0,1,axis='columns').sort_index(axis=1)
    continent = continent.loc[idx,:] 
    return df, continent

continent_colors = {'Asia':'yellow', 'Europe':'green', 'Africa':'brown',
                    'Oceania':'blue', 'Americas':'red'}
df, continent = get_data()
years = df.columns.levels[0]

app = dash.Dash()
app.layout = html.Div(children=[
    html.H3(children='World Stats'),

    html.Div('Move the mouse over a bubble to get information about the country'), 

    html.Div([
        html.Div([ dcc.Graph(id='main-graph'), ], style={'width':'90%', }),

        html.Div([
            dcc.Checklist(
                id='crossfilter-which-continent',
                options=[{'label': i, 'value': i} for i in sorted(continent_colors.keys())],
                values=sorted(continent_colors.keys()),
                labelStyle={'display':'block'},
            ),
            html.P(id='placeholder'), # used when a callback should not act on another component
            html.Div('X scale'),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Log',
                labelStyle={'display':'block'},
            )
        ], style={'width': '10%', 'float':'right'}),
    ], style={
        'padding': '0px 50px', 
        'display':'flex',
        'justifyContent':'center'
    }),

    html.Div([
            dcc.Slider(
                id='crossfilter-year-slider',
                min=years[0],
                max=years[-1],
                value=years[0],
                step = 1,
                marks={str(year): str(year) for year in years[::5]},
            ),
            dcc.Interval(
                id='auto-stepper',
                interval=60*60*1000, # in milliseconds
                n_intervals=0    # change by itself every interval
            ),
            html.Button(
                STOP,   # for some reason loading the page makes a click!
                id='button-start-stop', 
                style={'margin-left':'30'},
            ),
    ], style={
        'padding': '0px 50px', 
        'display':'flex',
        'justifyContent':'center'
    }),

    html.P(),
    html.Div(id='div-country'),

    html.Div([
        dcc.Graph(id='x-time-series', 
                  style={'width':'33%', 'display':'inline-block'}),
        dcc.Graph(id='y-time-series',
                  style={'width':'33%', 'display':'inline-block', 'padding-left': '0.5%'}),
        dcc.Graph(id='pop-time-series',
                  style={'width':'33%', 'display':'inline-block', 'padding-left': '0.5%'}),
    ], style={ 'display':'flex', 'justifyContent':'center', }),

], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(240, 240, 240)',
         'padding': '10px 50px 10px 50px',
         }
)

def traces(df, which_continent, year):
    res = []
    for c in which_continent:
        dfc = df[year][continent.region == c]
        res.append(
            go.Scatter(
                 x = dfc['incomes'],
                 y = dfc['children'],
                 mode = 'markers',
                 marker = dict( size = np.sqrt(dfc['population']/1E5),
                                color = continent_colors[c],
                              ),
                 text = dfc.index)
        )
    return res

@app.callback(
    dash.dependencies.Output('main-graph', 'figure'),
    [ dash.dependencies.Input('crossfilter-which-continent', 'values'),
      dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
      dash.dependencies.Input('crossfilter-year-slider', 'value')])
def update_graph(which_continent, xaxis_type, year):
    return {
        'data': traces(df, which_continent, year),
        'layout': go.Layout(
            xaxis = dict(title='GDP per Capita (US $)',
                         type= 'linear' if xaxis_type == 'Linear' else 'log',
                         range=(0,55000) if xaxis_type == 'Linear' 
                                         else (np.log10(50), np.log10(55000)) 
                        ),
            yaxis = dict(title='Child per woman', range=(0,9)),
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest',
            showlegend=False,
        )
    }


def create_time_series(country, what, axis_type, title):
    return {
        'data': [go.Scatter(
            x=years,
            y=df.loc[country, (years, what)],
            mode='lines+markers',
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 50, 'b': 20, 'r': 10, 't': 20},
            'yaxis': {'title':title,
                      'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


def get_country(hoverData):
    if hoverData == None:  # init value
        return df.index.values[np.random.randint(len(df))]
    return hoverData['points'][0]['text']

@app.callback(
    dash.dependencies.Output('div-country', 'children'),
    [dash.dependencies.Input('main-graph', 'hoverData')])
def country_chosen(hoverData):
    return get_country(hoverData)

# graph incomes vs years
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('main-graph', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_type):
    country = get_country(hoverData)
    return create_time_series(country, 'incomes', xaxis_type, 'GDP per Capita (US $)')

# graph children vs years
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('main-graph', 'hoverData'),])
def update_x_timeseries(hoverData):
    country = get_country(hoverData)
    return create_time_series(country, 'children', 'linear', 'Child per woman')

# graph population vs years
@app.callback(
    dash.dependencies.Output('pop-time-series', 'figure'),
    [dash.dependencies.Input('main-graph', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_pop_timeseries(hoverData, xaxis_type):
    country = get_country(hoverData)
    return create_time_series(country, 'population', xaxis_type, 'Population')

# start and stop the movie
@app.callback(
    dash.dependencies.Output('button-start-stop', 'children'),
    [dash.dependencies.Input('button-start-stop', 'n_clicks')],
    [dash.dependencies.State('button-start-stop', 'children')]) # note that 3rd field
def button_on_click(n_clicks, text):
    if text == START:
        return STOP
    else:
        return START

# this one is triggered by the previous one because we cannot have 2 outputs
# in the same callback
@app.callback(
    dash.dependencies.Output('auto-stepper', 'interval'),
    [dash.dependencies.Input('button-start-stop', 'children')])
def button_on_click(text):
    if text == START:    # then it means we are stopped
        return 60*60*1000  # just one event an hour
    else:
        return 0.5*1000 

# see if it should move the slider for simulating a movie
@app.callback(
    dash.dependencies.Output('crossfilter-year-slider', 'value'),
    [dash.dependencies.Input('auto-stepper', 'n_intervals')],
    [dash.dependencies.State('crossfilter-year-slider', 'value'),
     dash.dependencies.State('button-start-stop', 'children')]) 
def on_interval(n_intervals, year, text):
    if text == STOP:  # then we are running
        if year == years[-1]:
            return years[0]
        else:
            return year + 1
    else:
        return year  # nothing changes


if __name__ == '__main__':
    app.run_server()


{{ PreviousNext("40 -- A dashboard with Dash -- Layout.ipynb", "")}}